In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from selenium import webdriver

**Ejercicios de de Web Scraping.**

### Objetivos 
* web scraping.
* Documentar datos recogidos con web scraping.

# Ejercicio 1

Realice web scraping de dos de las tres páginas web propuestas utilizando **BeautifulSoup** primero y **Selenium** después. 

* http://quotes.toscrape.com

* https://www.bolsamadrid.es

* www.wikipedia.es (haga alguna búsqueda primero y escrapea algún contenido)


# Solución

### 1.1 BeautifulSoup (web scraping)

A continuación vamos a hacer web scraping de alguna página de Wikipedia. Por ejemplo, los juegos olímpicos de Río de Janeiro 2016.

1. Introduzca la URL de la página para descargarla y recuperarla:

In [7]:
URL='https://es.wikipedia.org/wiki/Juegos_Ol%C3%ADmpicos_de_R%C3%ADo_de_Janeiro_2016'

2. Haga una petición al servidor utilizando la URL anterior:

In [8]:
response=requests.get(URL)
response

<Response [200]>

3. Analice la página descargada:

In [9]:
page = BeautifulSoup(response.text, 'html.parser')

2. Obtenga el título de la página. Compruebe que es el mismo que aparece en el navegador.

In [10]:
page.title

<title>Juegos Olímpicos de Río de Janeiro 2016 - Wikipedia, la enciclopedia libre</title>

3. La siguiente captura de pantalla muestra la tabla que se va a extraer.

![rio2016](rio2016.png)

Existe la posibilidad de que la página contenga más de una tabla, lo cual es común en las páginas de Wikipedia. Por este motivo, tenemos que mirar todas las tablas y encontrar la que vamos a trabajar. 

Con el método **.get('class')**, vamos a obtener todas las tablas y sus respectivas clases de estilos.

In [11]:
for tab in page.find_all('table'):
    print(tab.get('class'))

['infobox']
None
['wikitable']
None
['wikitable', 'sortable']
['toccolours']
['wikitable']
['wikitable', 'striped', 'sortable', 'col1cen', 'col2izq', 'col3der', 'col4der', 'col5der', 'col6der']
['wikitable']
['hlist', 'navbox-inner']


Para nuestra tabla su respectiva clase es **striped**. 

4. obtenemos nuestra tabla con el método **find()**, al cual le pasamos como parámetrso, el **tag**(etiqueta HTML) y la clase del tag **striped**.

In [12]:
tab = page.find('table', class_ ='striped')
print(tab.tbody) # imprime el cuerpo de la tabla

<tbody><tr>
<th>Núm.
</th>
<th>País
</th>
<th><span typeof="mw:File"><a class="mw-file-description" href="/wiki/Archivo:Gold_medal_olympic.svg" title="Oro"><img alt="Oro" class="mw-file-element" data-file-height="300" data-file-width="300" decoding="async" height="18" src="//upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Gold_medal_olympic.svg/18px-Gold_medal_olympic.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Gold_medal_olympic.svg/27px-Gold_medal_olympic.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/4/4f/Gold_medal_olympic.svg/36px-Gold_medal_olympic.svg.png 2x" width="18"/></a></span>
</th>
<th><span typeof="mw:File"><a class="mw-file-description" href="/wiki/Archivo:Silver_medal_olympic.svg" title="Plata"><img alt="Plata" class="mw-file-element" data-file-height="300" data-file-width="300" decoding="async" height="18" src="//upload.wikimedia.org/wikipedia/commons/thumb/6/67/Silver_medal_olympic.svg/18px-Silver_medal_olympic.svg.png" srcset

5. El siguiente código imprime los datos de nuestra tabla: 

In [13]:
data = []
for row in tab.find_all('tr'):
    row_data = []
    for col in row.find_all('td'):
        row_data.append(col.get_text().replace('\n',''))
    data.append(row_data)
data

[[],
 ['1', ' Estados Unidos\xa0(USA)', '46', '37', '38', '121'],
 ['2', ' Reino Unido\xa0(GBR)', '27', '23', '17', '67'],
 ['3', ' China\xa0(CHN)', '26', '18', '26', '70'],
 ['4', ' Rusia\xa0(RUS)', '19', '18', '19', '56'],
 ['5', ' Alemania\xa0(GER)', '17', '10', '15', '42'],
 ['6', ' Japón\xa0(JPN)', '12', '8', '21', '41'],
 ['7', ' Francia\xa0(FRA)', '10', '18', '14', '42'],
 ['8', ' Corea del Sur\xa0(KOR)', '9', '3', '9', '21'],
 ['9', ' Italia\xa0(ITA)', '8', '12', '8', '28'],
 ['10', ' Australia\xa0(AUS)', '8', '11', '10', '29'],
 ['13', ' Brasil\xa0(BRA)', '7', '6', '6', '19']]

#### ¿Comó funciona?
* Creamos un lista vacía llamada **data**, la cual estará compuesta por las filas.
* Con un bucle *for* encontramos todas las filas &lt;tr&gt;&lt;/tr&gt; 
* Creamos un lista vacía de filas llamada **row_data**
* con un bucle *for* encontramos todas las columnas &lt;td&gt;&lt;/td&gt; 
* Con el método **.get_text()**, añadimos el texto que se encuentra dentro de cada &lt;td&gt;&lt;/td&gt;  a la lista **row_data**
* Por último, añadimos **row_data** a la lista **data**

In [14]:
# obtener el primer elemento de data
data[0:1]

[[]]

5. Con la libreria **pandas** creaamos nuestro dataframe con **data**. Aquí vamos a obviar el eleemnto **data[0:1]**, pues está vacío.

In [16]:
df_bs4=pd.DataFrame(data[1:],columns=['ranking','country','gold_medal','silver_medal','bronze_medal','total'])
df_bs4

,ranking,country,gold_medal,silver_medal,bronze_medal,total
0,1,Estados Unidos (USA),46,37,38,121
1,2,Reino Unido (GBR),27,23,17,67
2,3,China (CHN),26,18,26,70
3,4,Rusia (RUS),19,18,19,56
4,5,Alemania (GER),17,10,15,42
5,6,Japón (JPN),12,8,21,41
6,7,Francia (FRA),10,18,14,42
7,8,Corea del Sur (KOR),9,3,9,21
8,9,Italia (ITA),8,12,8,28
9,10,Australia (AUS),8,11,10,29


### 1.2 Selenium (web scraping)

A continuación haremos web scraping de la página https://www.bolsamadrid.es. En dicha página vamos a obtener la tabla de los precios de las acciones de los mercados de España.

1. Crear una instancia de Chrome, esto abre una ventana de Chrome localmente. Este comando le permite realizar pruebas hasta que utilice el método **.quit()** para finalizar la conexión al navegador.

In [81]:
browser = webdriver.Chrome()

2. Utilizamos el método **.get()** del controlador para cargar el sitio web. 

In [82]:
browser.get('https://www.bolsasymercados.es/bme-exchange/es/Mercados-y-Cotizaciones/Acciones/Mercado-Continuo/Precios/ibex-35-ES0SI0000005')

3. Navegamos a nuestra página, utilizando el método **.find_elements_by_xpath** para recuperar los elementos **tr** de nuestra tabla (ver gráfico).


![acciones](acciones.png)

Al copiar el **xpath** obtenemos lo siguiente: **/html/body/main/div/div/div/div[2]/div[5]/table/tbody/tr[1]** pues es el primer **tr[1]** que hemos seleccionado. Pero para obtener todos, solo pasaremos como parámetro **/html/body/main/div/div/div/div[2]/div[5]/table/tbody/tr** y lo haremos de la siguiente manera:

In [83]:
table = browser.find_elements_by_xpath("/html/body/main/div/div/div/div[2]/div[5]/table/tbody/tr")
#registers = len(browser.find_elements_by_xpath("/html/body/main/div/div/div/div[2]/div[5]/table/tbody/tr"))

4. Para encontrar los **tds** utilizamos el método **.find_element_by_tag_name**. El texto resultante de cada **td** se obtiene con el método **.text**.

In [84]:
data_table=[]
for trs in table:
    tds = trs.find_elements_by_tag_name('td')
    rows_table=[]
    for td in tds:
        rows_table.append(td.text)
    data_table.append(rows_table)

5. Por último, creamos nuestro DataFrame.

In [85]:
df=pd.DataFrame(data_table,columns =['Nombre', 'ultimo', 'porcentaje_Dif','maximo','minimo','volumen','efectivo','fecha','hora'])
df

,Nombre,ultimo,porcentaje_Dif,maximo,minimo,volumen,efectivo,fecha,hora
0,ACCIONA,"127,8500","-1,65%","130,7000","127,8000",55.381,"7.120,89",04/09/2023,Cierre
1,ACCIONA ENER,"26,1200","-3,04%","27,0000","26,0800",338.507,"8.911,92",04/09/2023,Cierre
2,ACERINOX,"9,3740","0,43%","9,4580","9,3340",418.903,"3.932,85",04/09/2023,Cierre
3,ACS,"32,3700","0,12%","32,6400","32,2600",170.582,"5.528,16",04/09/2023,Cierre
4,AENA,"145,0500","0,42%","147,0500","144,5500",95.925,"13.927,46",04/09/2023,Cierre
5,AMADEUS,"63,2800","0,19%","63,9400","63,1400",291.184,"18.448,43",04/09/2023,Cierre
6,ARCELORMIT.,"24,9550","-0,56%","25,4500","24,8600",148.926,"3.747,54",04/09/2023,Cierre
7,B.SANTANDER,"3,5150","-0,13%","3,5640","3,5015",17.339.932,"61.245,95",04/09/2023,Cierre
8,BA.SABADELL,"1,0610","-0,24%","1,0825","1,0580",14.320.813,"15.278,59",04/09/2023,Cierre
9,BANKINTER,"5,8600","-0,54%","5,9460","5,8400",1.750.461,"10.282,83",04/09/2023,Cierre


6. Cerramos el navegador.

In [97]:
browser.quit()

# Ejercicio 2

Documente en un Word tu conjunto de datos generado con la información que tienen los distintos archivos de Kaggle.

Para saber más

A modo de ejemplo de lo que se pide puede consultar este enlace:

https://www.kaggle.com/datasets/vivovinco/20212022-football-team-stats .

# Solución

1. ### Ver el archivo Ejercicio2.pdf

# Ejercicio 3
Elige una página web que quieras y realiza web scraping mediante la librería **Selenium** primero y **Scrapy** después. 

## 3.1 Selenium (web scraping)

### Mundial de fútbol femenino

En el siguiente ejercicio utilzaremos la libreria **selenium** para extraer en un dataframe todos los partidos que se disputaron en el mundial de fútbol femenino del 2023.

# Solución

In [93]:
browser = webdriver.Chrome()

In [25]:
browser.get('https://en.wikipedia.org/wiki/2023_FIFA_Women%27s_World_Cup')
ele1=browser.find_elements_by_xpath('//th[@class="fhome"]')
ele2=browser.find_elements_by_xpath('//th[@class="faway"]')
ele3=browser.find_elements_by_xpath('//th[@class="fscore"]')

In [29]:
home=[]
away=[]
score=[]
for i,j,k in zip(ele1,ele2,ele3):
    home.append(i.text)
    away.append(j.text)
    score.append(k.text)

In [28]:
df_women_cup=pd.DataFrame({'home_team':home,'away_team':away,'score':score})
df_women_cup['year']=2023
df_women_cup

,home_team,away_team,score,year
0,New Zealand,Norway,1–0,2023
1,Philippines,Switzerland,0–2,2023
2,New Zealand,Philippines,0–1,2023
3,Switzerland,Norway,0–0,2023
4,Switzerland,New Zealand,0–0,2023
...,...,...,...,...
59,England,Colombia,2–1,2023
60,Spain,Sweden,2–1,2023
61,Australia,England,1–3,2023
62,Sweden,Australia,2–0,2023


### Todos los mundiales de fútbol femenino

El propósito del siguiente ejemplo es extraer de forma dinámica todo los mundiales de fútbol femenino celebrados desde 1991 hasta 2023

1. Creamos una lista con los años de celebración de cada mundial, tener en cuenta que estos se realizan cada cuatro años.

In [90]:
years = [i for i in range(1991,2027,4)]
years

[1991, 1995, 1999, 2003, 2007, 2011, 2015, 2019, 2023]

2. Creamos una función **matches** cuyo parámetro será el año de celebración de cada mundial. Además, utilizaremos **f-string** con la posibilidad de definir string y dentro hacer la sustitución de variables, estás variables van dentro de llaves.

In [94]:
def matches(year):
    url=f'https://en.wikipedia.org/wiki/{year}_FIFA_Women%27s_World_Cup'
    browser.get(url)
    ele1=browser.find_elements_by_xpath('//th[@class="fhome"]')
    ele2=browser.find_elements_by_xpath('//th[@class="faway"]')
    ele3=browser.find_elements_by_xpath('//th[@class="fscore"]')
    home=[]
    away=[]
    score=[]
    for i,j,k in zip(ele1,ele2,ele3):
        home.append(i.text)
        away.append(j.text)
        score.append(k.text)
    df_soccer=pd.DataFrame({'home_team':home,'away_team':away,'score':score})
    df_soccer['year']=year
    return  df_soccer

3. Iteramos la función **matches** por cada año de la lista **years**.

In [95]:
#print(matches(1999))
match=[matches(year) for year in years]
df_matches=pd.concat(match,ignore_index=True)
df_matches

,home_team,away_team,score,year
0,China,Norway,4–0,1991
1,Denmark,New Zealand,3–0,1991
2,Norway,New Zealand,4–0,1991
3,China,Denmark,2–2,1991
4,China,New Zealand,4–1,1991
...,...,...,...,...
343,England,Colombia,2–1,2023
344,Spain,Sweden,2–1,2023
345,Australia,England,1–3,2023
346,Sweden,Australia,2–0,2023


4. Por último, exportamos nuestro Dataset a un archivo **CSV**.

In [96]:
df_matches.to_csv('women_soccer_world_cup.csv',index=False)

## 3.2 Scrapy

### La solución se encuentra en la carpeta Matches junto con los archivos matches.csv y matches.json.

### A continuación se hará un resumen de los pasos a seguir para la solución del ejercicio.

1. Para crear un nuevo proyecto, simplemente ejecutamos el siguiente comando:

``` python
scrapy startproject Matches
```

Se creará el siguiente árbol de archivos:

```js
├── Matches
│   ├── Matches
│   │   ├── __init__.py
│   │   ├── items.py
│   │   ├── middlewares.py
│   │   ├── pipelines.py
│   │   ├── settings.py
│   │   └── spiders
│   │       └── __init__.py
│   └── scrapy.cfg
```

2. Ingresamos a la carpeta **Matches**

```py
cd Matches
```

3. Ejecutamos el siguiente comando:

```python
scrapy genspider matches en.wikipedia.org/wiki/2023_FIFA_Women%27s_World_Cup
```

El comando anterior genera un achivo python **matches.py** dentro de la carpeta **spiders**. Este archivos junto con **items.py** serán los que utilizaremos para realizar el scraping. 

```python
├── Matches
│   ├── Matches
│   │   ├── __init__.py
│   │   ├── items.py
│   │   ├── middlewares.py
│   │   ├── pipelines.py
│   │   ├── __pycache__
│   │   │   ├── __init__.cpython-39.pyc
│   │   │   └── settings.cpython-39.pyc
│   │   ├── settings.py
│   │   └── spiders
│   │       ├── __init__.py
│   │       ├── matches.py
│   │       └── __pycache__
│   │           └── __init__.cpython-39.pyc
│   └── scrapy.cfg

```

4. Inspeccionar archivo **matches.py**.

```python
import scrapy


class MatchesSpider(scrapy.Spider):
    name = 'matches'
    allowed_domains = ['en.wikipedia.org/wiki/2023_FIFA_Women%27s_World_Cup']
    start_urls = ['http://en.wikipedia.org/wiki/2023_FIFA_Women%27s_World_Cup/']

    def parse(self, response):
        pass
```

5. Antes de continuar vamos a inspecionar las tablas de nuestra web. Podemos ver que todas las etiquetas &lt;th&gt; dentro del &lt;tr&gt;(ver imagen) de nuestra tabla tienen algo en comun, la class="fhome", la class="fscore" y la class="faway" que representan el equipo local, resultado y equipo visitante respectivamente.

![xpath](xpath.png)

6. Editar el archivo python **items.py** para obtener nuestras columnas a las cuales llamaremos **home**, **away** y **score**. 

```python
import scrapy


class MatchesItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()

    home = scrapy.Field()    
    away = scrapy.Field()
    score = scrapy.Field()
    pass
```

7. Editar **matches.py** para poder realizar el scraping.

  * En el archivo matches.py importamos el archivo items.py 
  * Capturamos las clases a través de response.xpath(), por ejemplo, response.xpath('//th[@class="fhome"]')
  * Instanciamos el método MatchesItem() al cual hemos llamaremos item.
  * Obtenemos el texto para cada item, por ejemplo, para obtener el texto de item['home']  navegamos hacía la clase fhome, luego a su &lt;span&gt; y por ultimo a la etiqueta &lt;a&gt; 
  * Con yield item, descargamos todos los items.

```python
import scrapy

from Matches.items import MatchesItem
class MatchesSpider(scrapy.Spider):
    name = 'matches'
    allowed_domains = ['en.wikipedia.org/wiki/2023_FIFA_Women%27s_World_Cup']
    start_urls = ['https://en.wikipedia.org/wiki/2023_FIFA_Women%27s_World_Cup']

    def parse(self, response):
        rows_home = response.xpath('//th[@class="fhome"]')
        rows_away = response.xpath('//th[@class="faway"]')
        rows_score = response.xpath('//th[@class="fscore"]')
        for row_home, row_away, row_score in zip(rows_home,rows_away,rows_score):
    
            item = MatchesItem()
            item['home']= row_home.xpath('./span/a//text()').extract()                        
            item['away']= row_away.xpath('./span/a//text()').extract()
            item['score']= row_score.xpath('./a//text()').extract()
    
            yield item     
          
        pass
```

#### Observación

Añadir al archivo **settings.py** la sigueinte línea:
```python
FEED_EXPORT_FIELDS=['home','away','score']
```
para poder mantener el orden que se ha asignado a cada columna, de lo contrario las columnas se imprimirán en orden alfabético ('away','home','score'). 

Este problema ocurre cuando extraemos los datos a un archivo **CSV**

8. Comando para extraer datos:

```python
scrapy crawl matches
```

9. Comandos para exportar datos a un archivo csv o json:

```python
scrapy crawl matches -o matches.csv
```

```json
scrapy crawl matches -o matches.json
```